In [ ]:
# 📦 Install necessary libraries
!pip install transformers xgboost pandas scikit-learn --quiet

In [ ]:
# 📚 Imports
import pandas as pd
import numpy as np
from datetime import datetime
import calendar
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from transformers import pipeline, set_seed

In [13]:
# 🔗 Load real-time dataset from Google Sheet
dataset_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQgUPQua_FhZsD9u3Bp69NMSR1oF2sWaH7LfzCBnYqeHvlbNAXYztTR9GEgYsRVIdGQRzbICb81jUIZ/pub?output=csv"
df = pd.read_csv(dataset_url)

In [14]:
#  Data Preprocessing
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Month'] = df['Order Date'].dt.month
df['Year'] = df['Order Date'].dt.year
df['Season'] = df['Month'].apply(lambda x: (
    'Winter' if x in [12, 1, 2] else
    'Spring' if x in [3, 4, 5] else
    'Summer' if x in [6, 7, 8] else
    'Autumn'
))
df['Product Category'] = df['Product Name'].apply(
    lambda name: 'Decor' if 'decor' in name.lower() else (
        'Cool Drink' if 'drink' in name.lower() or 'cool' in name.lower() else 'Other'
    )
)

In [15]:
# Simulated Local LLM for Seasonal Reasoning
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
def seasonal_reasoning(month, category):
    prompt = f"In the month of {calendar.month_name[month]}, how will sales of {category} products likely behave?"
    result = generator(prompt, max_length=50, num_return_sequences=1)
    return result[0]['generated_text']

Device set to use cpu


In [ ]:
# Add contextual feature using the LLM output
df['LLM_Insight'] = df.apply(lambda row: seasonal_reasoning(row['Month'], row['Product Category']), axis=1)
df['LLM_Weight'] = df['LLM_Insight'].apply(lambda x: 1.2 if 'increase' in x.lower() else 0.9)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for 

In [17]:
# ✅ Checkpoint: Import required packages again just to be safe
import pandas as pd

# ✅ Step 1: Make sure 'Order Date' is datetime
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')

# ✅ Step 2: Make sure 'Sales' is numeric
df['Sales'] = pd.to_numeric(df['Sales'], errors='coerce')

# ✅ Step 3: Sort values to ensure shifting works properly
df = df.sort_values(by=['Product Name', 'Order Date'])

# ✅ Step 4: Perform the shift
print("Shifting sales to create 'Last Year Sales' feature...")
df['Last Year Sales'] = df.groupby('Product Name')['Sales'].shift(12)

# ✅ Step 5: Drop rows with NaNs in required columns
df = df.dropna(subset=['Sales', 'Last Year Sales'])

# ✅ Step 6: Add LLM_Weight if missing
if 'LLM_Weight' not in df.columns:
    print("LLM_Weight not found. Assigning default weight = 1.")
    df['LLM_Weight'] = 1

# ✅ Step 7: Extract time features
df['Month'] = df['Order Date'].dt.month
df['Year'] = df['Order Date'].dt.year

# ✅ Step 8: Create feature matrix and target
features = ['Month', 'Year', 'LLM_Weight', 'Last Year Sales']
X = df[features]
y = df['Sales']

# ✅ Print sample for quick check
print("✅ Preprocessing complete. Feature preview:")
print(X.head())


Shifting sales to create 'Last Year Sales' feature...
✅ Preprocessing complete. Feature preview:
      Month  Year  LLM_Weight  Last Year Sales
1947     11  2020         0.9           2323.0
781      11  2020         0.9           4663.0
1883     12  2020         0.9           3299.0
538       1  2021         0.9           4488.0
1980      1  2021         0.9           3510.0


<ipython-input-17-7432023fb094>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month'] = df['Order Date'].dt.month
<ipython-input-17-7432023fb094>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year'] = df['Order Date'].dt.year


In [18]:
df['Last Year Sales'] = df.groupby('Product Name')['Sales'].shift(12)
df = df.dropna()
features = ['Month', 'Year', 'LLM_Weight', 'Last Year Sales']
X = df[features]
y = df['Sales']

In [19]:
# 📈 Train/Test Split and XGBoost Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100)
model.fit(X_train, y_train)
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print(f"✅ Model Trained | RMSE: {rmse:.2f}")

✅ Model Trained | RMSE: 1727.88


In [20]:
from datetime import datetime

# Step 1: Get current month and year
current_month = datetime.now().month
current_year = datetime.now().year

# Step 2: Get latest row per product to predict future sales
future_df = df.groupby('Product Name').tail(1).copy()
future_df['Month'] = current_month
future_df['Year'] = current_year

# Step 3: Generate LLM insight + weight
future_df['LLM_Insight'] = future_df.apply(lambda row: seasonal_reasoning(current_month, row['Product Category']), axis=1)
future_df['LLM_Weight'] = future_df['LLM_Insight'].apply(lambda x: 1.2 if 'increase' in x.lower() else 0.9)

# Step 4: Get sales from same month last year
last_year_sales = (
    df[(df['Month'] == current_month) & (df['Year'] == current_year - 1)]
    .groupby('Product Name')['Sales']
    .last()
    .reset_index()
    .rename(columns={'Sales': 'Last Year Sales'})
)

# Step 5: Merge with current product info
future_df = future_df.merge(last_year_sales, on='Product Name', how='left')

# 🔒 Ensure 'Last Year Sales' exists before using
if 'Last Year Sales' not in future_df.columns:
    future_df['Last Year Sales'] = df['Sales'].mean()
else:
    future_df['Last Year Sales'] = future_df['Last Year Sales'].fillna(df['Sales'].mean())

# Step 6: Define feature set safely
features = ['Month', 'Year', 'LLM_Weight', 'Last Year Sales']
future_features = future_df[features]

# Step 7: Make predictions
future_df['Predicted Sales'] = model.predict(future_features)

# Step 8: Show results
display(future_df[['Product Name', 'Month', 'Year', 'Predicted Sales', 'LLM_Insight']])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

,Product Name,Month,Year,Predicted Sales,LLM_Insight
0,Air Purifier,4,2025,2987.790039,"In the month of April, how will sales of Other..."
1,Bluetooth Speaker,4,2025,2987.790039,"In the month of April, how will sales of Other..."
2,Cool Drink,4,2025,2987.790039,"In the month of April, how will sales of Cool ..."
3,Decorative Light,4,2025,2987.790039,"In the month of April, how will sales of Decor..."
4,Electric Toothbrush,4,2025,2987.790039,"In the month of April, how will sales of Other..."
5,Espresso Machine,4,2025,2987.790039,"In the month of April, how will sales of Other..."
6,Flower Vase,4,2025,2987.790039,"In the month of April, how will sales of Other..."
7,LED TV,4,2025,2987.790039,"In the month of April, how will sales of Other..."
8,Laptop Stand,4,2025,2987.790039,"In the month of April, how will sales of Other..."
9,Noise Cancelling Headphones,4,2025,2987.790039,"In the month of April, how will sales of Other..."
